# Imports

In [1]:
import pandas as pd
import re
import os


# lectura

# funciones

In [2]:
# 

root_excels = "C:\\Users\\hsakurada\\Desktop\\ESTUDIO\\wea\\CMS-PRUEBA-TECNICA\\descargas"

MESES = {
    "enero": 1, "febrero": 2, "marzo": 3, "abril": 4,
    "mayo": 5, "junio": 6, "julio": 7, "agosto": 8,
    "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12
}
años = ['2025','2024','2023','2022','2021','2020','2019','2018','2017','2016','2015','2014','2013','2012','2011','2010']

In [3]:

def extraer_mes_nomb_excel(nombre_archivo):
    # quitar extensión
    nombre = os.path.splitext(nombre_archivo)[0].lower()

    # separar por espacios, guiones, underscores, etc.
    partes = re.split(r"[ _\-]+", nombre)

    # buscar coincidencia exacta
    for parte in partes:
        if parte in MESES:
            return parte  # retorna el nombre del mes encontrado

    # búsqueda más flexible: " - diciembre 2012"
    for mes in MESES:
        if mes in nombre:
            return mes

    return None


def extraer_año(nombre_archivo):
    
    # quitar extensión
    nombre = os.path.splitext(nombre_archivo)[0].lower()

    # separar por espacios, guiones, underscores, etc.
    partes = re.split(r"[ _\-]+", nombre)

    # buscar coincidencia exacta
    for parte in partes:
        if parte in años:
            return parte  # retorna el año encontrado

    # búsqueda más flexible: " - diciembre 2012"
    for año in años:
        if año in nombre:
            return año

    return None

import os

def listar_todos_los_root_excels(root_excels):
    excels = []
    for root, dirs, files in os.walk(root_excels):
        for file in files:
            if file.lower().endswith(('.xlsx', '.xls')):
                excels.append(os.path.join(root, file))
    return excels



In [4]:
rutas = listar_todos_los_root_excels(root_excels)
rutas[:4]

['C:\\Users\\hsakurada\\Desktop\\ESTUDIO\\wea\\CMS-PRUEBA-TECNICA\\descargas\\2020\\Base de Datos de Transporte Público - Abril 2021.xlsx',
 'C:\\Users\\hsakurada\\Desktop\\ESTUDIO\\wea\\CMS-PRUEBA-TECNICA\\descargas\\2020\\Base de Datos de Transporte Público - Agosto 2021.xlsx',
 'C:\\Users\\hsakurada\\Desktop\\ESTUDIO\\wea\\CMS-PRUEBA-TECNICA\\descargas\\2020\\Base de Datos de Transporte Público - Diciembre 2021.xlsx',
 'C:\\Users\\hsakurada\\Desktop\\ESTUDIO\\wea\\CMS-PRUEBA-TECNICA\\descargas\\2020\\Base de Datos de Transporte Público - Enero 2021.xlsx']

In [5]:
import os
import re
import unicodedata
import pandas as pd
from rapidfuzz import fuzz

# ============================================================
# PRECOMPILAR REGEX (20x más rápido si se usa miles de veces)
# ============================================================
_re_non_alnum = re.compile(r"[^0-9a-z]+")
_re_multi_unders = re.compile(r"_+")

def normalize_fast(text: str) -> str:
    """
    Normaliza texto MUY rápido: sin acentos, minúsculas, underscores limpios.
    """
    if not isinstance(text, str):
        return text

    text = unicodedata.normalize("NFKD", text)
    text = "".join(c for c in text if not unicodedata.combining(c))
    text = text.lower()

    text = _re_non_alnum.sub("_", text)
    text = _re_multi_unders.sub("_", text)
    return text.strip("_")


# ============================================================
# NORMALIZACIÓN DE COLUMNAS / HOJAS (ULTRA RÁPIDO)
# ============================================================
def normalize_sheet_name(name):
    return normalize_fast(name)

def normalize_column_name(name):
    return normalize_fast(name)


# ============================================================
# UNIFICADOR DE COLUMNAS (OPTIMIZADO)
# ============================================================
def unify_column_names_fast(dfs, cutoff=0.75):
    """
    Mucho más rápido que difflib.
    - Agrupa por hash prefix → fuzzy matching solo cuando necesario.
    """

    # 1) recolectar todas las columnas normalizadas
    col_master = set()
    normalized_per_df = []

    for df in dfs:
        norm_cols = [normalize_column_name(c) for c in df.columns]
        normalized_per_df.append(norm_cols)
        col_master.update(norm_cols)

    col_master = list(col_master)

    # 2) bucketing por primeros 4 caracteres (reduce fuzzy 80%)
    buckets = {}
    for c in col_master:
        key = c[:4]               # bucket rápido, casi cero colisiones
        buckets.setdefault(key, []).append(c)

    # 3) renombrar cada df
    results = []
    for df, cols_norm in zip(dfs, normalized_per_df):
        df_new = df.copy()
        rename_map = {}

        for orig, norm in zip(df.columns, cols_norm):
            bucket = buckets.get(norm[:4], [])

            # si hay match exacto → directo
            if norm in bucket:
                rename_map[orig] = norm
                continue

            # fuzzy matching SOLO si hay pocos candidatos
            best = None
            best_score = cutoff
            for cand in bucket:
                score = fuzz.ratio(norm, cand) / 100
                if score > best_score:
                    best_score = score
                    best = cand

            rename_map[orig] = best if best else norm

        df_new = df_new.rename(columns=rename_map)
        results.append(df_new)

    return results


# ============================================================
# LECTURA Y CONCATENACIÓN POR HOJA (OPTIMIZADO)
# ========

# ============================================================
# LECTURA Y CONCATENACIÓN POR HOJA (OPTIMIZADO + DEBUG)
# ============================================================
def leer_y_concatenar_por_hoja(lista_rutas, join="intersection", fuzzy_cutoff=0.6, verbose=True):
    grupos = {}  # nombre hoja normalizada → lista dfs

    # ======================================================
    # 1) LECTURA DE ARCHIVOS Y AGRUPACIÓN POR HOJA
    # ======================================================
    for ruta in lista_rutas:
        nombre_archivo = os.path.basename(ruta)
        if verbose:
            print(f"\n📄 Archivo detectado: {nombre_archivo}")

        mes = extraer_mes_nomb_excel(nombre_archivo)
        año = extraer_año(nombre_archivo)

        try:
            xls = pd.ExcelFile(ruta)
            if verbose:
                print(f"   → Contiene hojas: {xls.sheet_names}")
        except Exception as e:
            if verbose:
                print(f"   ❌ Error al abrir: {e}")
            continue

        for hoja in xls.sheet_names:
            hoja_norm = normalize_sheet_name(hoja)

            if verbose:
                print(f"   📝 Leyendo hoja '{hoja}' → normalizada como '{hoja_norm}'")

            try:
                df = xls.parse(hoja)
            except Exception as e:
                if verbose:
                    print(f"      ❌ Error al leer hoja '{hoja}': {e}")
                continue

            df = df.copy()
            df["mes"] = mes
            df["año"] = año

            # agrega df al grupo
            grupos.setdefault(hoja_norm, []).append(df)

            if verbose:
                print(f"      ✔ Agregada al grupo '{hoja_norm}'. Tamaño actual: {len(grupos[hoja_norm])}")

    # ======================================================
    # 2) PROCESAR CADA GRUPO
    # ======================================================
    resultados = {}

    for hoja_norm, lista_df in grupos.items():
        if verbose:
            print(f"\n🔷 Procesando grupo '{hoja_norm}' con {len(lista_df)} hojas...")

        # 2.1 Normalizar columnas
        if verbose:
            print(f"   🔍 Normalizando columnas (fuzzy cutoff={fuzzy_cutoff})…")

        lista_df_ren = unify_column_names_fast(lista_df, cutoff=fuzzy_cutoff)

        # Mostrar columnas normalizadas comparativamente
        if verbose:
            for i, (df_before, df_after) in enumerate(zip(lista_df, lista_df_ren), start=1):
                print(f"\n   📑 Archivo {i}: columnas originales → normalizadas:")
                print(f"     Original : {list(df_before.columns)}")
                print(f"     Normaliz.: {list(df_after.columns)}")

        # 2.2 Sacar universo de columnas
        col_sets = [set(df.columns) for df in lista_df_ren]

        if join == "intersection":
            columnas_finales = sorted(set.intersection(*col_sets))
            if verbose:
                print(f"\n   🔗 Usando INTERSECCIÓN de columnas:")
        else:
            columnas_finales = sorted(set.union(*col_sets))
            if verbose:
                print(f"\n   🔗 Usando UNIÓN de columnas:")

        if verbose:
            print(f"      → Columnas finales: {columnas_finales}")

        # 2.3 Alinear DataFrames
        dfs_alineados = [df.reindex(columns=columnas_finales) for df in lista_df_ren]

        # 2.4 Concatenar
        if verbose:
            print(f"\n   📚 Concatenando {len(dfs_alineados)} DataFrames…")

        df_final = pd.concat(dfs_alineados, ignore_index=True)

        if verbose:
            print(f"   ✔ Grupo '{hoja_norm}' concatenado. Filas totales: {len(df_final)}")

        resultados[hoja_norm] = df_final

    return resultados


In [6]:
import os
import re
import unicodedata
import pandas as pd
from rapidfuzz import fuzz, process as fuzzprocess

# ============================================================
# PRECOMPILAR REGEX (20x más rápido)
# ============================================================
_re_non_alnum = re.compile(r"[^0-9a-z]+")
_re_multi_unders = re.compile(r"_+")

# ============================================================
# NORMALIZACIÓN RÁPIDA
# ============================================================
def normalize_fast(text: str) -> str:
    """Normaliza texto MUY rápido: sin acentos, minúsculas, underscores limpios."""
    if not isinstance(text, str):
        return text

    text = unicodedata.normalize("NFKD", text)
    text = "".join(c for c in text if not unicodedata.combining(c))
    text = text.lower()

    text = _re_non_alnum.sub("_", text)
    text = _re_multi_unders.sub("_", text)
    return text.strip("_")


def normalize_sheet_name(name):
    """Normaliza y singulariza para evitar 'vehiculos' / 'vehiculo'."""
    norm = normalize_fast(name)
    # singularización simple
    if norm.endswith("es") and len(norm) > 3:
        norm = norm[:-2]
    elif norm.endswith("s") and len(norm) > 2:
        norm = norm[:-1]
    return norm


def normalize_column_name(name):
    return normalize_fast(name)


# ============================================================
# UNIFICADOR DE COLUMNAS (OPTIMIZADO)
# ============================================================
def unify_column_names_fast(dfs, cutoff=0.75):
    """
    Unifica nombres de columnas con:
    - normalización
    - bucketing por prefijo (reduce fuzzy match 80%)
    """

    # --------------- recolectar columnas normalizadas ---------------
    col_master = set()
    normalized_per_df = []

    for df in dfs:
        norm_cols = [normalize_column_name(c) for c in df.columns]
        normalized_per_df.append(norm_cols)
        col_master.update(norm_cols)

    col_master = list(col_master)

    # --------------- buckets por primeros 4 caracteres ---------------
    buckets = {}
    for col in col_master:
        key = col[:4]
        buckets.setdefault(key, []).append(col)

    # --------------- renombrar DataFrames ---------------
    results = []
    for df, norm_cols in zip(dfs, normalized_per_df):
        rename_map = {}
        for orig, norm in zip(df.columns, norm_cols):
            candidates = buckets.get(norm[:4], [])

            if norm in candidates:
                rename_map[orig] = norm
                continue

            # fuzzy solo cuando necesario
            best, score = fuzzprocess.extractOne(norm, candidates, scorer=fuzz.ratio) or (None, 0)
            rename_map[orig] = best if score >= cutoff * 100 else norm

        results.append(df.rename(columns=rename_map))

    return results


# ============================================================
# AGRUPADOR DE HOJAS SIMILARES (vehiculo / vehiculos / vehículo)
# ============================================================
def agrupar_nombres_hoja(sheet_names, cutoff=80):
    """
    Une nombres similares en un solo grupo:
    Ej: vehiculo, vehiculos, vehículo → vehiculo
    """
    grupos = {}
    canonicos = []

    for name in sheet_names:
        norm = normalize_sheet_name(name)

        if canonicos:
            match = fuzzprocess.extractOne(norm, canonicos, scorer=fuzz.ratio)
            if match and match[1] >= cutoff:
                grupos[match[0]].append(name)
                continue

        grupos[norm] = [name]
        canonicos.append(norm)

    return grupos


# ============================================================
# LECTURA GLOBAL
# ============================================================
def leer_y_concatenar_por_hoja(lista_rutas, join="intersection", fuzzy_cutoff=0.6, verbose=True):
    grupos = {}  # hoja_normalizada : list_dfs

    # ======================================================
    # 1) PRIMER PASO: Recolectar todas las hojas para agrupar
    # ======================================================
    if verbose:
        print("\n🔍 Analizando nombres de hojas para detectar equivalencias...")

    all_sheet_names = []
    file_sheets = {}

    for ruta in lista_rutas:
        try:
            xls = pd.ExcelFile(ruta)
            file_sheets[ruta] = xls.sheet_names
            all_sheet_names.extend(xls.sheet_names)

            if verbose:
                print(f"   📄 {os.path.basename(ruta)} → hojas: {xls.sheet_names}")

        except Exception as e:
            if verbose: print(f"   ❌ Error leyendo {ruta}: {e}")
            continue

    equivalencias = agrupar_nombres_hoja(all_sheet_names, cutoff=80)

    if verbose:
        print("\n📌 Grupos de nombres de hojas detectados:")
        for canon, items in equivalencias.items():
            print(f"   - {canon}: {items}")

    # ======================================================
    # 2) LECTURA DE ARCHIVOS USANDO LOS GRUPOS DEFINIDOS
    # ======================================================
    for ruta in lista_rutas:
        nombre_archivo = os.path.basename(ruta)
        if verbose:
            print(f"\n📂 Procesando archivo: {nombre_archivo}")

        try:
            xls = pd.ExcelFile(ruta)
        except Exception as e:
            if verbose: print(f"   ❌ Error abriendo archivo: {e}")
            continue

        for hoja_original in xls.sheet_names:
            hoja_norm = normalize_sheet_name(hoja_original)

            # buscar canonical del grupo
            hoja_final = hoja_norm
            for canon, miembros in equivalencias.items():
                if hoja_original in miembros or hoja_norm == canon:
                    hoja_final = canon
                    break

            if verbose:
                print(f"   📝 Leyendo hoja '{hoja_original}' → grupo '{hoja_final}'")

            try:
                df = xls.parse(hoja_original).copy()
            except Exception as e:
                if verbose: print(f"      ❌ Error al leer hoja: {e}")
                continue
            
            # agregar mes y año
            mes = extraer_mes_nomb_excel(nombre_archivo)
            año = extraer_año(nombre_archivo)
            df["mes"] = mes
            df["año"] = año

            if verbose:
                print(f"      ➕ Columnas agregadas: mes={mes}, año={año}")
                print(f"      📐 Tamaño DF: {df.shape}")

            grupos.setdefault(hoja_final, []).append(df)

    # ======================================================
    # 3) PROCESAR CADA GRUPO
    # ======================================================
    resultados = {}

    for hoja_norm, lista_df in grupos.items():
        if verbose:
            print(f"\n🔷 Procesando grupo '{hoja_norm}' ({len(lista_df)} DataFrames)…")

        # 1) normalizar columnas
        if verbose:
            print("   🔧 Normalizando nombres de columnas...")

        lista_df_norm = unify_column_names_fast(lista_df, cutoff=fuzzy_cutoff)

        # 2) inter/union de columnas
        col_sets = [set(df.columns) for df in lista_df_norm]

        columnas_finales = (
            sorted(set.intersection(*col_sets))
            if join == "intersection"
            else sorted(set.union(*col_sets))
        )

        if verbose:
            print(f"   🧩 Columnas finales ({len(columnas_finales)}):")
            print(f"      {columnas_finales}")

        # 3) alinear
        if verbose:
            print("   📏 Alineando columnas...")

        dfs_alineados = [df.reindex(columns=columnas_finales) for df in lista_df_norm]

        # 4) concatenar
        df_final = pd.concat(dfs_alineados, ignore_index=True)
        resultados[hoja_norm] = df_final

        if verbose:
            print(f"   ✔ Concatenado completo: {len(df_final)} filas finales")

    return resultados



In [7]:
bases = leer_y_concatenar_por_hoja(rutas)

# ejemplo: imprimir tamaños de cada base
for columnas, df in bases.items():
    print(columnas, df.shape)





🔍 Analizando nombres de hojas para detectar equivalencias...
   📄 Base de Datos de Transporte Público - Abril 2021.xlsx → hojas: ['Servicios', 'Replegal', 'Vehículos', 'O-D Recorridos', 'Trazados']
   📄 Base de Datos de Transporte Público - Agosto 2021.xlsx → hojas: ['Servicios', 'RepLegales', 'Vehículo', 'O-D Recorrido', 'Trazados']
   📄 Base de Datos de Transporte Público - Diciembre 2021.xlsx → hojas: ['Servicios', 'RepLegales', 'Vehículo', 'O-D Recorridos', 'Trazado']
   📄 Base de Datos de Transporte Público - Enero 2021.xlsx → hojas: ['Servicios', 'RepLegales', 'Vehiculos', 'O-D Recorridos', 'Trazados']
   📄 Base de Datos de Transporte Público - Febrero 2021.xlsx → hojas: ['Servicio', 'RepLegales', 'Vehículo', 'O-D Recorrido', 'Trazado']
   📄 Base de Datos de Transporte Público - Julio 2021.xlsx → hojas: ['Servicios', 'Representantes Legales', 'Vehículos', 'O-D Recorridos', 'Trazados']
   📄 Base de Datos de Transporte Público - Junio 2021.xlsx → hojas: ['Servicios', 'RepLegales',

KeyboardInterrupt: 